In the [previous lesson](https://www.kaggle.com/prashantkikani/is-it-a-cat-or-dog-convolutional-nn/), we learned basics of computer vision & CNN(convolutional neural network) by building a classifier with hands-on Python code experience.

Today in this lesson, we will learn the basics of Natural Language Processing(NLP) by building a model on "Sarcasm Detection".<br><br>
![sarcasm](https://memegenerator.net/img/instances/75322005/was-that-sarcasm.jpg)

We will use LSTM (Long Short Term Memory) layers to create our deep learning model who can predict whether a statement is sarcastic or not.<br><br>

But before going to start the coding part of the lesson, let's first understand basics of few things that we are going to use.<br>

## Tokenize
In NLP, we work with text data.<br>
But, machines / computers can't read text. They just deal with numbers.<br>
So, to convert text data into numbers, we give a unique ID number to each word & replace word it's corresponding number.
### That's what Tokenizer does.
It converts our text data to numeric format.<br>
Like, sentence `"machine learning is cool"` will convert into something like `"2315 834 12 4510"`.<br><br>

After converting our text data into numbers, now, we can use that data to train our ML models.<br>
But, there's one thing we are missing.<br><br>

After converting, each word into numbers, `king` word will converted into something like 1678, `queen` to something like 56832, `man` to something like 4285, `woman` to something like 6387 etc. (These numbers I have choosen randomly. Point being, it'll converted to numbers).<br>

* How, we will give our model information like, `king` & `queen` are related same as `man` & `woman`.<br>
* Or, `italy`, `spain`, `brazil`, `india`, `japan` etc. words have one thing in common. (They all are name of countris)<br>
* Or, `cricket`, `football`, `tennis` etc. words are also similar. (names of games)<br>
* Or, the way words `china` & `beijing` are related, words `russia` & `moscow` are also related in the same way.<br><br>

All these are knowledge related things.<br>
How can we give this knowledge to our NLP models so that, it can use that knowledge to perform better at detecting sarcasm?<br>

Answer is `word embeddings`.

## Word embeddings

Word embeddings are nothing but 1D vector of length 200-500, containing floating point numbers.<br>
Each word have corresponding 1D array for it.<br><br>

For e.g.<br>
`king` : `[-0.34, 0.98, -0.04, 0.32]`<br>
`queen` : `[0.25, -0.23, -0.10, -0.72]`<br>
.<br>
.<br>
.<br>

Here it's just 4 numbers in a 1D vector, but in actual there will be 200-500 numbers in a 1D vector for each word.<br>

### Why we use word embeddings?
These word embeddings captures intrinsic meaning in some way for each words.<br>
It kind of stores “meaning” of words or “insights” of the words inside them.<br>
For e.g.<br>
![gender](https://blog.acolyer.org/wp-content/uploads/2016/04/word2vec-gender-relation.png)
<center>The way man & woman are related, king & queen, uncle & aunt are related the same way. </center><br>

We can even perform mathematical operations on word embeddings & get expected results.<br>
![gender maths](http://jalammar.github.io/images/word2vec/king-analogy-viz.png)
<center>If we subtract word embeddings of `man` from `king` & add word embedding of `woman`, we get word embedding of `queen`</center><br>

![word embeddings country](https://blog.acolyer.org/wp-content/uploads/2016/04/word2vec-dr-fig-2.png)
<center>Relation between country & capitals</center><br>

### We use word embeddings to transfer the knowledge of words to model so that it can perform better.

### How do we get these word embeddings?
* There's something called [Language Models](https://en.wikipedia.org/wiki/Language_model).<br>
* They basically learn to predict next word of the sentence given all the past words.<br>
* We will not go there as of now. If you want to learn more about it, I recommend reading [this blog](https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/).

So, now we have understood tokenizer & word embeddings.<br>
Let's also quickly understand LSTM.<br>

## LSTM (Long Short Term Memory)

LSTM is type of RNN (Recurrent Neural Network) which "remembers" text to perform better in the task we are doing.<br>
For e.g. <br>
To answer the question, `"Bob is from France. Alice is from Japan. So, what's the mother-tongue of Bob?"`<br>
We need to remember the first sentence, `"Bob is from France"`.<br>
Without knowing / remembering that, we can't answer the question `"French"`<br><br>

Same way, LSTM remembers, past text to perform better at the task we are doing.<br>

I highly recommend to check out [this blog](https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714) to understand LSTM. <br>That blog contains a simple explanation of LSTM that almost everyone can understand.<br>

Here's what a cell of LSTM looks like.<br>
![LSTM](https://miro.medium.com/max/700/1*laH0_xXEkFE0lKJu54gkFQ.png)
<center>An LSTM cell</center>
<br>

There are 3 valves in a single LSTM cell.<br>
1. **Forget valve**
    * This valve decides what should be forgetted from the past text. What things are irrelevant to remember.
    * First part from left is forget valve.
2. **Memory valve**
    * This valve decides how much new memory should influence the old memory.
    * New memory is generated by a single layer neural network with pentagon shaped activation function(tanh).
    * Output will be element-wise multiple the new memory valve, and add to the old memory to form the new memory.
    * Memory valve is in the middle of the above figure.
3. **Output valve**
    * This valve generates output of this perticular LSTM cell.
    * It's in the right part of above LSTM cell figure.
    * It's controlled by new memory & previous output.
    * This valve controls how much new memory should output to the next LSTM unit.
<br>

### In one LSTM layer, there will be a chain of this type of LSTM cells. 

So, that's enough of theoratical talking.<br>
Let's do it by code.

In [ ]:
# pandas to open data files & processing it.
import pandas as pd
# to see all columns
pd.set_option('display.max_columns', None)
# To see whole text
pd.set_option('max_colwidth', -1)

# numpy for numeric data processing
import numpy as np

# keras for deep learning model creation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.utils import plot_model

# to fix random seeds
import random
import tensorflow as tf
import torch
import os

# Regular Expression for text cleaning
import re

# to track the progress - progress bar
from tqdm.notebook import tqdm

Let's open the data files now.

In [ ]:
sarcasm_data = pd.read_csv("../input/sarcasm/train-balanced-sarcasm.csv")
print(sarcasm_data.shape)
sarcasm_data.head()

We just need `comment` & `label` column.<br>
So, let's remove others.

In [ ]:
sarcasm_data.drop(['author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'], axis=1, inplace=True)
# remove empty rows
sarcasm_data.dropna(inplace=True)
sarcasm_data.head()

In [ ]:
sarcasm_data['label'].value_counts()

So, there are `505368` sentences with sarcastic news headlines !!<br>

Let's do some pre-processing on our text data.<br>
These are the common practices which can improve performance in almost any NLP task.<br><br>

One common thing we can do is to remove `contractions`.<br>
### Like, "ain't" to "is not", "can't" to "can not" etc.

In [ ]:
mispell_dict = {"ain't": "is not", "cannot": "can not", "aren't": "are not", "can't": "can not", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",
                "doesn't": "does not",
                "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did",
                "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
                "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "so's": "so as", "this's": "this is", "that'd": "that would",
                "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is", "they'd": "they would",
                "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have",
                "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "wont": "will not", "won't've": "will not have", "would've": "would have",
                "wouldn't": "would not",
                "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color',
                'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
                'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What',
                'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I',
                'theBest': 'the best', 'howdoes': 'how does', 'Etherium': 'Ethereum',
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what',
                'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

mispell_dict = {k.lower(): v.lower() for k, v in mispell_dict.items()}

Let's make our preprocessing function.

In [ ]:
def preprocessing_text(s):
    # making our string lowercase & removing extra spaces
    s = str(s).lower().strip()
    
    # remove contractions.
    s = " ".join([mispell_dict[word] if word in mispell_dict.keys() else word for word in s.split()])
    
    # removing \n
    s = re.sub('\n', '', s)
    
    # put spaces before & after punctuations to make words seprate. Like "king?" to "king", "?".
    s = re.sub(r"([?!,+=—&%\'\";:¿।।।|\(\){}\[\]//])", r" \1 ", s)
    
    # Remove more than 2 continues spaces with 1 space.
    s = re.sub('[ ]{2,}', ' ', s).strip()
    
    return s

In [ ]:
# apply preprocessing_text function
sarcasm_data['comment'] = sarcasm_data['comment'].apply(preprocessing_text)
sarcasm_data.head()

In [ ]:
# total unique words we are going to use.
TOTAL_WORDS = 40000

# max number of words one sentence can have
MAX_LEN = 50

# width of of 1D embedding vector
EMBEDDING_SIZE = 300

In [ ]:
%%time
tokenizer = Tokenizer(num_words=TOTAL_WORDS)
tokenizer.fit_on_texts(list(sarcasm_data['comment']))

train_data = tokenizer.texts_to_sequences(sarcasm_data['comment'])
train_data = pad_sequences(train_data, maxlen = MAX_LEN)
target = sarcasm_data['label']

Let's open embedding file now & store in a matrix.

In [ ]:
%%time
EMBEDDING_FILE = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in tqdm(open(EMBEDDING_FILE)))

word_index = tokenizer.word_index
nb_words = min(TOTAL_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_SIZE))

In [ ]:
for word, i in tqdm(word_index.items()):
    if i >= TOTAL_WORDS: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

Let's build our NLP deep learning model now..

In [ ]:
embedding_matrix.shape

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    tf.random.set_seed(seed)

# We fix all the random seed so that, we can reproduce the results.
seed_everything(2020)

In [ ]:
input_layer = Input(shape=(MAX_LEN,))

embedding_layer = Embedding(TOTAL_WORDS, EMBEDDING_SIZE, weights = [embedding_matrix])(input_layer)

LSTM_layer = Bidirectional(LSTM(128, return_sequences = True))(embedding_layer)
maxpool_layer = GlobalMaxPool1D()(LSTM_layer)

dense_layer_1 = Dense(64, activation="relu")(maxpool_layer)
dropout_1 = Dropout(0.5)(dense_layer_1)

dense_layer_2 = Dense(32, activation="relu")(dropout_1)
dropout_2 = Dropout(0.5)(dense_layer_2)

output_layer = Dense(1, activation="sigmoid")(dropout_2)

model = Model(input=input_layer, output=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Before training the model first, let's understand our model first.

`input_layer` : Input layer with which we will get text sentence.<br><br>
`embedding_layer` : Embedding layer with which we will map each word with it's corresponding embedding vector.<br><br>
`LSTM_layer` : LSTM layer with 128 LSTM cells.
* We are using Bidirectional to run LSTM from both side of the text sentence.
    1. Left to Right
    2. Right to Left
* Purpose of this is to give our model both side context.
* It's also possible to not use this. But using this have provided good results.
<br><br>
`maxpool_layer` : Max pool layer is used to minimize the image size by pooling maximum number out of 2x2 grid.
![maxpool](https://distilledai.com/wp-content/uploads/2020/04/2x2-max-pool-CNN.png)
<br><br>
`dense_layer_1` : Feed-forward dense layer to classify the features captured by LSTM layer.
<br><br>
`dropout_1` : Dropout is interesting trick. In Dropout, we randomly turn off some percentage of our neurons so that their's output can't go to next layer. Here we are turning off 20% of our total neurons.
* Purpose of doing this is again to make our training robust.
* Network should not depend some specific neurons to make predictions. And random turn will allow us to do that.
* Picture below help us to understand it. 
![dropout](https://distilledai.com/wp-content/uploads/2020/04/dropout-in-deep-learning.png)
<br><br>
`dense_layer_2` & `dropout_2` are same as above.
<br><br>
`output_layer` : To get the output prediction from the neural network.


In [ ]:
plot_model(model, show_shapes=True)

## Let's start the training of our model

In [ ]:
BATCH_SIZE = 512
EPOCHS = 2

history = model.fit(
    train_data, target,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    # We are using randomly selected 20% sentences as validation data.
    validation_split=0.2
)

### Cool !!

Training is over.<br>
We can see validation accuracy above.<br>

### Let's test our model on some random input now.

In [ ]:
sarcasm_data[sarcasm_data['label']==1].sample(20)

In [ ]:
sentence = "sun rises from the east"
sentence = preprocessing_text(sentence)
print(sentence)

sentence = tokenizer.texts_to_sequences([sentence])
sentence = pad_sequences(sentence, maxlen = MAX_LEN)
sentence

We can see, how our sentence got converted into numbers.<br>

## We are doing padding to keep the final length same for every sentence no matter the sentence length.
## Our Neural network have learned to ignore 0 in the training itself.

In [ ]:
# Make the prediction.
prediction = model.predict(sentence)
prediction[0][0]

In [ ]:
print("So, it's saying sentence have probability of %.3f percent"%(prediction[0][0]*100))

In [ ]:
sentence = "Isn't it great that, your girlfriend dumped you?"
sentence = preprocessing_text(sentence)
print(sentence)

sentence = tokenizer.texts_to_sequences([sentence])
sentence = pad_sequences(sentence, maxlen = MAX_LEN)
sentence

In [ ]:
# Make the prediction.
prediction = model.predict(sentence)
prediction[0][0]

In [ ]:
print("So, it's saying sentence have probability of %.3f percent"%(prediction[0][0]*100))

## Sarcasm is a subjective matter.<br> One person can call a sentence sarcastic & another don't.
### So, results may be subjective. But our goal was to understand some basic concepts of NLP.

# Summary

* We saw what Tokenization is.
* We saw what word embedings are.
* We understood how a LSTM cell works in a nutshell.
* We cleaned the text data to use it in training
* We trained a deep learning model

## Upvote this kernel if you have learned something from it.<br>
## Tell me if you have any kind of doubts / questions in comment section below.

## In the next lesson, we will learn basics of "Recommendation" by recommending similar Movies to users.

# See you in the [next lesson](https://www.kaggle.com/prashantkikani/can-you-recommend-me-a-movie-recommendation) !